## 데이터 변형

In [2]:
import pandas as pd
import numpy as np

rng = np.random.RandomState(1234)

### 계층적 인덱싱

In [2]:
ser = pd.Series(rng.randn(9), index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'], [1, 2, 3, 1, 3, 1, 2, 2, 3]])
ser

a  1    0.471435
   2   -1.190976
   3    1.432707
b  1   -0.312652
   3   -0.720589
c  1    0.887163
   2    0.859588
d  2   -0.636524
   3    0.015696
dtype: float64

In [3]:
ser.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [4]:
ser['b']

1   -0.312652
3   -0.720589
dtype: float64

In [5]:
ser['b':'c']

b  1   -0.312652
   3   -0.720589
c  1    0.887163
   2    0.859588
dtype: float64

In [6]:
ser.loc[['a','d']]

a  1    0.471435
   2   -1.190976
   3    1.432707
d  2   -0.636524
   3    0.015696
dtype: float64

In [7]:
ser.loc[:,2]

a   -1.190976
c    0.859588
d   -0.636524
dtype: float64

`unstack` 메소드를 이용해 시리즈를 데이터프레임으로 변형할 수 있다. unstack은 행 인덱스의 지정된 수준을 열 인덱스의 가장 안쪽 수준으로 변경하는 메소드이다. 

In [8]:
ser.unstack()

,1,2,3
a,0.471435,-1.190976,1.432707
b,-0.312652,NaN,-0.720589
c,0.887163,0.859588,NaN
d,NaN,-0.636524,0.015696


In [9]:
# 역으로 unstack 자료를 원래 자료로 stack을 이용해 만들 수 있다.
ser.unstack().stack()

a  1    0.471435
   2   -1.190976
   3    1.432707
b  1   -0.312652
   3   -0.720589
c  1    0.887163
   2    0.859588
d  2   -0.636524
   3    0.015696
dtype: float64

In [10]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [11]:
frame.index.names = ['key1','key2']

In [12]:
frame.columns.names = ['state','color']

In [13]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [14]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

다중인덱스 객체를 만들어서 필요할 때 재사용할 수 있다. 열 인덱스는 다음과 같이 다중 인덱스 객체의 from_arrays를 이용해서 만들수도 있다.

In [15]:
col_index = pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']], names=['state', 'color'])

In [17]:
col_index

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

In [18]:
pd.DataFrame(np.arange(4*3).reshape(4,3), columns = col_index)

state  Ohio     Colorado
color Green Red    Green
0         0   1        2
1         3   4        5
2         6   7        8
3         9  10       11

`from_product([[상위 수준 인덱스], [하위 수준 인덱스]])`는 상위 수준 인덱스 각각에 하위 수준 인덱스가 만들어 진다. 여기서는 'a' 상위 수준에 1, 2 하위 수준이 만들어지고 'b'에 대해서도 마찬가지이다.

In [19]:
row_index = pd.MultiIndex.from_product([['a', 'b'], [1, 2]], names=['key1', 'key2'])
row_index

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           names=['key1', 'key2'])

In [20]:
pd.DataFrame(np.arange(4 * 3).reshape(4, 3), index=row_index)


0   1   2
key1 key2           
a    1     0   1   2
     2     3   4   5
b    1     6   7   8
     2     9  10  11

In [21]:
pd.DataFrame(np.arange(4 * 3).reshape(4, 3), columns=col_index, index=row_index)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

### 수준별 정렬

`swaplevel`은 두 개의 수준을 인자로 받아 두 수준을 서로 바꾼 형태의 새로운 객체를 반환한다.

In [22]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

다중인덱스에서 `sort_index`를 사용하여 특정 수준에 대한 정렬을 시행할 수 있다.

In [23]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [24]:
frame.sort_index(axis='columns', level=1)

state     Colorado  Ohio    
color        Green Green Red
key1 key2                   
a    1           2     0   1
     2           5     3   4
b    1           8     6   7
     2          11     9  10

### 데이터프레임 열 인덱싱

In [25]:
frame = pd.DataFrame({'a': range(7),
                      'b': range(7,0,-1),
                      'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [26]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

drop=False 옵션을 이용해서 제거하지 않을 수도 있다.

In [27]:
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [28]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


### 데이터세트 결합

#### JOIN/Merge

In [29]:
df1 = pd.DataFrame({'A': ['A'+str(i) for i in range(4)]})
df1

,A
0,A0
1,A1
2,A2
3,A3


In [30]:
df2 = pd.DataFrame({'B': ['B'+str(i) for i in range(4)]})
df2

,B
0,B0
1,B1
2,B2
3,B3


In [31]:
pd.merge(df1, df2)

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [32]:
df3 = pd.DataFrame({'B': ['B'+str(i) for i in range(4)],
                    'C': ['C'+str(i) for i in range(4)]})
df3.loc[4] = ['BX', 'C4']
df3

,B,C
0,B0,C0
1,B1,C1
2,B2,C2
3,B3,C3
4,BX,C4


In [33]:
pd.merge(df2,df3)

,B,C
0,B0,C0
1,B1,C1
2,B2,C2
3,B3,C3


In [34]:
df4 = df2.copy()
df4.loc[3] = 'B0'
df4

,B
0,B0
1,B1
2,B2
3,B0


In [35]:
df5 = df3.copy()
df5.loc[4,'B'] = 'B0'
df5

,B,C
0,B0,C0
1,B1,C1
2,B2,C2
3,B3,C3
4,B0,C4


In [37]:
pd.merge(df4,df5)

,B,C
0,B0,C0
1,B0,C4
2,B0,C0
3,B0,C4
4,B1,C1
5,B2,C2


In [38]:
pd.merge(df4, df5, how='outer')

,B,C
0,B0,C0
1,B0,C4
2,B0,C0
3,B0,C4
4,B1,C1
5,B2,C2
6,B3,C3


In [39]:
pd.merge(df4, df5, how='left')

,B,C
0,B0,C0
1,B0,C4
2,B1,C1
3,B2,C2
4,B0,C0
5,B0,C4


In [40]:
pd.merge(df4, df5, how='right')

,B,C
0,B0,C0
1,B0,C0
2,B0,C4
3,B0,C4
4,B1,C1
5,B2,C2
6,B3,C3


In [41]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})

In [42]:
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})

In [43]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [44]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [45]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


중복되는 열이름이 있을 경우 `suffixes=` 선택인자를 이용해서 열이름 첨자를 지정할 수 있다. `suffixes`를 설정하지 않으면 왼쪽은 _x, 오른쪽은 _y를 붙인다.

In [46]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [47]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


#### 인덱스로 Merge

In [3]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [4]:
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
right1

,group_val
a,3.5
b,7.0


`right_index=True`는 오른쪽 데이터프레임의 병합키는 인덱스를 사용하고 `left_on='key'`는 왼쪽 병합키는 'key'를 사용한다는 뜻이다.

In [5]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [6]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                               'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])

In [7]:
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [8]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [9]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [10]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])

In [11]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [12]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [13]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [14]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [15]:
pd.merge(left2, right2, left_index=True, right_index=True, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [16]:
left2.join(right2)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
c,3.0,4.0,9.0,10.0
e,5.0,6.0,13.0,14.0


join 메소드의 how 기본값은 left이고 병합키는 기본적으로 둘 다 인덱스를 사용한다. 호출 데이터프레임(join 메소드를 호출하는 데이터프레임)은 다른 병합키를 사용할 수 있다.

다음은 left1의 키는 key 열을 사용하고 right1의 키는 인덱스를 사용한 예이다.

In [17]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [18]:

another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])

In [19]:
left2.join([right2,another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [20]:
left2.join([right2,another], how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0
b,NaN,NaN,7.0,8.0,NaN,NaN
d,NaN,NaN,11.0,12.0,NaN,NaN
f,NaN,NaN,NaN,NaN,16.0,17.0


#### concat

In [21]:
s1 = pd.Series([0,1], index = ['a','b'])
s2 = pd.Series([2,3,4], index = ['c','d','e'])
s3 = pd.Series([5,6], index = ['f','g'])

In [23]:
# 행 방향으로 차례로 시리즈들을 합친다.
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [24]:
# 열방향으로 자료들을 붙이기 위해서는 axis=1을 지정하면 된다.
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [25]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [26]:
pd.concat([s1,s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


In [30]:
res = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
res

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [31]:
res.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In [32]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [33]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                   columns=['one', 'two'])
df1

,one,two
a,0,1
b,2,3
c,4,5


In [34]:
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                   columns=['three', 'four'])
df2

,three,four
a,5,6
c,7,8


In [35]:
pd.concat([df1, df2], axis=1, keys=['key1', 'key2'])

key1      key2     
   one two three four
a    0   1   5.0  6.0
b    2   3   NaN  NaN
c    4   5   7.0  8.0

In [36]:
pd.concat({'key1': df1, 'key2': df2}, axis=1)

key1      key2     
   one two three four
a    0   1   5.0  6.0
b    2   3   NaN  NaN
c    4   5   7.0  8.0

In [37]:
pd.concat([df1, df2], axis=1, keys=['key1', 'key2'], names=['level0', 'level1'])

level0 key1      key2     
level1  one two three four
a         0   1   5.0  6.0
b         2   3   NaN  NaN
c         4   5   7.0  8.0

In [38]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df1

,a,b,c,d
0,-1.020579,-0.569484,0.401608,-0.430044
1,-0.492573,-0.273701,1.083664,0.118373
2,1.386250,1.176543,-0.112662,0.027135


In [39]:
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
df2

,b,d,a
0,1.968965,-2.440293,1.092198
1,-0.095644,-0.889268,0.827479


In [40]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,-1.020579,-0.569484,0.401608,-0.430044
1,-0.492573,-0.273701,1.083664,0.118373
2,1.386250,1.176543,-0.112662,0.027135
3,1.092198,1.968965,NaN,-2.440293
4,0.827479,-0.095644,NaN,-0.889268


#### 중복있는 데이터 결합

In [3]:
a = pd.Series([np.nan, 2.5, 0.0, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])
a

f    NaN
e    2.5
d    0.0
c    3.5
b    4.5
a    NaN
dtype: float64

In [4]:
b = pd.Series([0., np.nan, 2., np.nan, np.nan, 5.],
              index=['a', 'b', 'c', 'd', 'e', 'f'])
b

a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    5.0
dtype: float64

a 성분 중 NaN이 있으면 대응되는 위치의 b의 값으로 설정하는 것이다. 하지만 이것은 인덱스와 상관없이 배열의 위치로만 값을 대입하는 것을 알 수 있다.

In [5]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 0. , 3.5, 4.5, 5. ])

시리즈의 인덱스에 대응해서 값을 지정하려면 combine_first 메소드를 사용하면 된다.

In [6]:
b.combine_first(a)

a    0.0
b    4.5
c    2.0
d    0.0
e    2.5
f    5.0
dtype: float64

In [7]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [8]:
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [9]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


### Reshaping

#### 계층적 인덱싱을 이용한 변형

stack: 지정된 열 수준을 행의 가장 안쪽 수준으로 변경한다.  
unstack: 지정된 행 수준을 열의 가장 안쪽 수준으로 변경한다.

In [10]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


stack 은 열축(number)을 행축의 가장 안쪽으로 위치시킨다. 그러면서 열축이 없어지기 때문에 결과는 시리즈가 된다.

In [11]:
res = data.stack()
res

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

unstack을 이용하면 역으로 행축의 가장 안쪽 수준인 number를 열축으로 위치시킨다. 따라서 원래 모양으로 돌아온다.

In [12]:
res.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [29]:
df_multi_level = pd.DataFrame([[1.0, 2.0], [3.0, 4.0]],
                              index=['cat', 'dog'],
                              columns=pd.MultiIndex.from_arrays([['weight', 'height'], ['kg', 'm']], names=['parts', 'units']))
df_multi_level

parts,weight,height
units,kg,m
cat,1.0,2.0
dog,3.0,4.0


In [30]:
df_multi_level.stack()

parts      height  weight
    units                
cat kg        NaN     1.0
    m         2.0     NaN
dog kg        NaN     3.0
    m         4.0     NaN

`level= `인자를 이용해서 변경하고자하는 수준을 직접 지정할 수 있다. 수준은 위에서부터 0, 1, … 이다.

In [31]:
#바깥쪽 수준인 parts -> level=0
stck_df = df_multi_level.stack(level=0)
stck_df

units        kg    m
    parts           
cat height  NaN  2.0
    weight  1.0  NaN
dog height  NaN  4.0
    weight  3.0  NaN

In [32]:
# 가장 왼쪽 수준 parts -> level=0
stck_df.unstack(level=0)

units    kg         m     
        cat  dog  cat  dog
parts                     
height  NaN  NaN  2.0  4.0
weight  1.0  3.0  NaN  NaN

In [33]:
df_multi_level.stack(level='units')

parts      height  weight
    units                
cat kg        NaN     1.0
    m         2.0     NaN
dog kg        NaN     3.0
    m         4.0     NaN

In [35]:
s1 = pd.Series([0,1,2,3], index = ['a','b','c','d'])
s1

a    0
b    1
c    2
d    3
dtype: int64

In [36]:
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
s2

c    4
d    5
e    6
dtype: int64

In [37]:
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [38]:
# 가장 안쪽 수준
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


unstack한 것을 다시 stack 하면 NaN 값은 자동으로 제거한다.

In [40]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

하지만 dropna=False 인자를 건네면 NaN 값이 그대로 나오는 것을 알 수 있다.

In [41]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64